# Local Mini-Circuit-Cutting

In [1]:
import numpy as np
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import SparsePauliOp
from circuit_knitting.cutting.automated_cut_finding import (
    find_cuts,
    OptimizationParameters,
    DeviceConstraints,
)
from circuit_knitting.cutting import (
    cut_wires,
    expand_observables,
    partition_problem,
    generate_cutting_experiments,
)
import json
import time

def experiment_with_circuit(num_qubits, depth):
    circuit = random_circuit(num_qubits, depth, max_operands=2, seed=1242)
    
    # Adjust observable strings to match the number of qubits
    observable_strings = ["Z" + "I" * (num_qubits - 1),
                          "I" * (num_qubits // 2) + "Z" + "I" * (num_qubits // 2 - 1),
                          "I" * (num_qubits - 1) + "Z"]
    observable_strings = [obs if len(obs) == num_qubits else "I" * num_qubits for obs in observable_strings]
    observable = SparsePauliOp(observable_strings)

    optimization_settings = OptimizationParameters(seed=111)
    device_constraints = DeviceConstraints(qubits_per_subcircuit=4)

    cut_circuit, metadata = find_cuts(circuit, optimization_settings, device_constraints)
    print(f'Found solution using {len(metadata["cuts"])} cuts with a sampling overhead of {metadata["sampling_overhead"]}.')
    for cut in metadata["cuts"]:
        print(f"{cut[0]} at circuit instruction index {cut[1]}")

    qc_w_ancilla = cut_wires(cut_circuit)
    observables_expanded = expand_observables(observable.paulis, circuit, qc_w_ancilla)

    partitioned_problem = partition_problem(circuit=qc_w_ancilla, observables=observables_expanded)
    subcircuits = partitioned_problem.subcircuits
    subobservables = partitioned_problem.subobservables

    print(f"Sampling overhead: {np.prod([basis.overhead for basis in partitioned_problem.bases])}")

    subexperiments, coefficients = generate_cutting_experiments(circuits=subcircuits, observables=subobservables, num_samples=1000)
    print(f"{len(subexperiments[0]) + len(subexperiments[1])} total subexperiments to run on backend.")

    backend = AerSimulator()
    results = {}

    total_time = 0
    for subexperiment in subexperiments.values():
        for i, circuit in enumerate(subexperiment):
            print(f"Running subcircuit {i}...")
            transpiled_circuit = transpile(circuit, backend)
            subexperiment_start = time.time()
            result = backend.run(transpiled_circuit).result()
            total_time += time.time() - subexperiment_start
            counts = result.get_counts()
            results[i] = counts
            print(f"Subcircuit {i} took {time.time() - subexperiment_start} seconds to run.")

    print("Running original circuit...")
    original_start = time.time()
    original_counts = backend.run(circuit).result().get_counts()
    original_time = time.time() - original_start
    results["original"] = original_counts

    print(f"Original circuit took {original_time} seconds to run.")
    print(f"Cuts took {total_time} seconds to run.")

    with open("results.json", "w") as f:
        json.dump(results, f)

    return results, total_time, original_time

# Example usage with optimized parameters (8 qubits, depth 3, 1000 samples)
results, total_time, original_time = experiment_with_circuit(8, 4)

results, total_time, original_time

Found solution using 2 cuts with a sampling overhead of 14.128980038696172.
Gate Cut at circuit instruction index 13
Gate Cut at circuit instruction index 19
Sampling overhead: 14.128980038696168
68 total subexperiments to run on backend.
Running subcircuit 0...
Subcircuit 0 took 0.004372119903564453 seconds to run.
Running subcircuit 1...
Subcircuit 1 took 0.0012557506561279297 seconds to run.
Running subcircuit 2...
Subcircuit 2 took 0.0015749931335449219 seconds to run.
Running subcircuit 3...
Subcircuit 3 took 0.0012869834899902344 seconds to run.
Running subcircuit 4...
Subcircuit 4 took 0.0011260509490966797 seconds to run.
Running subcircuit 5...
Subcircuit 5 took 0.0011637210845947266 seconds to run.
Running subcircuit 6...
Subcircuit 6 took 0.001251220703125 seconds to run.
Running subcircuit 7...
Subcircuit 7 took 0.0010752677917480469 seconds to run.
Running subcircuit 8...
Subcircuit 8 took 0.0011510848999023438 seconds to run.
Running subcircuit 9...
Subcircuit 9 took 0.00

({0: {'0 0': 524, '0 1': 500},
  1: {'0 0': 503, '1 1': 521},
  2: {'1 1': 519, '0 0': 505},
  3: {'0 1': 523, '0 0': 501},
  4: {'0 0': 505, '0 1': 519},
  5: {'0 0': 501, '0 1': 523},
  6: {'0 1': 467, '0 0': 557},
  7: {'0 1': 516, '0 0': 508},
  8: {'0 0': 522, '0 1': 502},
  9: {'0 1': 530, '0 0': 494},
  10: {'1 1': 539, '0 0': 485},
  11: {'0 0': 500, '1 1': 524},
  12: {'0 0': 529, '0 1': 495},
  13: {'0 1': 502, '0 0': 522},
  14: {'0 0': 535, '1 1': 489},
  15: {'0 0': 513, '1 1': 511},
  16: {'0 0': 501, '0 1': 523},
  17: {'0 0': 522, '0 1': 502},
  18: {'0 0': 529, '1 1': 495},
  19: {'1 1': 514, '0 0': 510},
  20: {'0 0': 483, '0 1': 541},
  21: {'0 0': 515, '0 1': 509},
  22: {'0 0': 506, '1 1': 518},
  23: {'1 1': 495, '0 0': 529},
  24: {'0 0': 480, '0 1': 544},
  25: {'0 1': 532, '0 0': 492},
  26: {'0 0': 512, '0 1': 512},
  27: {'0 0': 524, '0 1': 500},
  28: {'0 1': 534, '0 0': 490},
  29: {'0 0': 509, '0 1': 515},
  30: {'0 0': 540, '1 1': 484},
  31: {'0 0': 496,

# With IBMQ Mini-Circuit-Cutting 

In [2]:
import numpy as np
import json
import time
from qiskit import transpile, QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Session
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.random import random_circuit
from circuit_knitting.cutting.automated_cut_finding import (
    find_cuts,
    OptimizationParameters,
    DeviceConstraints,
)
from circuit_knitting.cutting import (
    cut_wires,
    expand_observables,
    partition_problem,
    generate_cutting_experiments,
)

def experiment_with_circuit(num_qubits, depth, use_ibmq=False):
    circuit = random_circuit(num_qubits, depth, max_operands=2, seed=1242)
    
    # Adjust observable strings to match the number of qubits
    observable_strings = ["Z" + "I" * (num_qubits - 1),
                          "I" * (num_qubits // 2) + "Z" + "I" * (num_qubits // 2 - 1),
                          "I" * (num_qubits - 1) + "Z"]
    observable_strings = [obs if len(obs) == num_qubits else "I" * num_qubits for obs in observable_strings]
    observable = SparsePauliOp(observable_strings)

    optimization_settings = OptimizationParameters(seed=111)
    device_constraints = DeviceConstraints(qubits_per_subcircuit=7)

    cut_circuit, metadata = find_cuts(circuit, optimization_settings, device_constraints)
    print(f'Found solution using {len(metadata["cuts"])} cuts with a sampling overhead of {metadata["sampling_overhead"]}.')
    for cut in metadata["cuts"]:
        print(f"{cut[0]} at circuit instruction index {cut[1]}")

    qc_w_ancilla = cut_wires(cut_circuit)
    observables_expanded = expand_observables(observable.paulis, circuit, qc_w_ancilla)

    partitioned_problem = partition_problem(circuit=qc_w_ancilla, observables=observables_expanded)
    subcircuits = partitioned_problem.subcircuits
    subobservables = partitioned_problem.subobservables

    print(f"Sampling overhead: {np.prod([basis.overhead for basis in partitioned_problem.bases])}")

    subexperiments, coefficients = generate_cutting_experiments(circuits=subcircuits, observables=subobservables, num_samples=1000)
    print(f"{len(subexperiments[0]) + len(subexperiments[1])} total subexperiments to run on backend.")
    
    print("Subexperiments:", subexperiments)

    results = {}
    total_time = 0

    if use_ibmq:
        # IBM Quantum setup
        service = QiskitRuntimeService(channel="ibm_quantum", token="f4578dfc2792a51df5907e166130b002695c4de4f2c694baa77b060328ac5dec1638d0930e8ebf00927adcfc57c660c7a1d8ce26b2ed7113230f6c23914599bf")
        backend = service.backend("ibm_brisbane")
        
        # Create a session with the ibm_brisbane backend
        session = Session(service=service, backend='ibm_brisbane')
        sampler = Sampler(session=session)
        
        for batch, subcircuits in subexperiments.items():
            transpiled_subcircuits = []
            for subcircuit in subcircuits:
                # Add classical register and measurement
                creg = ClassicalRegister(subcircuit.num_qubits, 'creg')
                subcircuit.add_register(creg)
                for q in range(subcircuit.num_qubits):
                    subcircuit.measure(q, creg[q])
                # Transpile the subcircuit
                transpiled_subcircuit = transpile(subcircuit, backend, optimization_level=1)
                transpiled_subcircuits.append(transpiled_subcircuit)
            
            subexperiment_start = time.time()
            job = sampler.run(transpiled_subcircuits)
            result = job.result()
            total_time += time.time() - subexperiment_start
            
            for i, counts in enumerate(result.quasi_dists):
                results[f"{batch}_{i}"] = counts
            
            print(f"Batch {batch} took {time.time() - subexperiment_start} seconds to run.")

        print("Running original circuit...")
        creg = ClassicalRegister(circuit.num_qubits, 'creg')
        circuit.add_register(creg)
        for q in range(circuit.num_qubits):
            circuit.measure(q, creg[q])
        transpiled_circuit = transpile(circuit, backend, optimization_level=1)
        
        original_start = time.time()
        job = sampler.run(transpiled_circuit, shots=1000)
        result = job.result()
        original_time = time.time() - original_start
        results["original"] = result.quasi_dists[0]

        for subexperiment in subexperiments.values():
            for i, circuit in enumerate(subexperiment):
                print(f"Running subcircuit {i}...")
                transpiled_circuit = transpile(circuit, backend)
                subexperiment_start = time.time()
                result = backend.run(transpiled_circuit).result()
                total_time += time.time() - subexperiment_start
                counts = result.get_counts()
                results[i] = counts
                print(f"Subcircuit {i} took {time.time() - subexperiment_start} seconds to run.")

        print("Running original circuit...")
        original_start = time.time()
        original_counts = backend.run(circuit).result().get_counts()
        original_time = time.time() - original_start
        results["original"] = original_counts

    print(f"Original circuit took {original_time} seconds to run.")
    print(f"Cuts took {total_time} seconds to run.")

    with open("results.json", "w") as f:
        json.dump(results, f)

    return results, total_time, original_time


# For IBM Quantum hardware:
results, total_time, original_time = experiment_with_circuit(8, 4, use_ibmq=True)


Found solution using 1 cuts with a sampling overhead of 1.783275571221539.
Gate Cut at circuit instruction index 13
Sampling overhead: 1.783275571221539
12 total subexperiments to run on backend.
Subexperiments: {0: [<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x319c13dd0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x16a550190>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x319c13a90>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x319c10cd0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x319b9e350>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x16a5314d0>], 1: [<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x16a5cd7d0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x16a493c10>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x1110f4510>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x16a4662d0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x16a47c350>, <qiskit.

/var/folders/w5/qtgfh_kx375_3385y0kbwgy40000gn/T/ipykernel_6724/3653426976.py:62: DeprecationWarning: The Sampler and Estimator V1 primitives have been deprecated as of qiskit-ibm-runtime 0.23.0 and will be removed no sooner than 3 months after the release date. Please use the V2 Primitives. See the `V2 migration guide <https://docs.quantum.ibm.com/api/migration-guides/v2-primitives>`_. for more details
  sampler = Sampler(session=session)


Batch 0 took 291.1208908557892 seconds to run.
Batch 1 took 63.281394958496094 seconds to run.
Running original circuit...
Running subcircuit 0...


/var/folders/w5/qtgfh_kx375_3385y0kbwgy40000gn/T/ipykernel_6724/3653426976.py:104: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/api/migration-guides/qiskit-runtime.
  result = backend.run(transpiled_circuit).result()


Subcircuit 0 took 98.17586588859558 seconds to run.
Running subcircuit 1...
Subcircuit 1 took 27.412906885147095 seconds to run.
Running subcircuit 2...
Subcircuit 2 took 20.862608909606934 seconds to run.
Running subcircuit 3...
Subcircuit 3 took 28.183209896087646 seconds to run.
Running subcircuit 4...
Subcircuit 4 took 35.09509873390198 seconds to run.
Running subcircuit 5...
Subcircuit 5 took 20.407121181488037 seconds to run.
Running subcircuit 0...
Subcircuit 0 took 39.35172772407532 seconds to run.
Running subcircuit 1...
Subcircuit 1 took 21.97625494003296 seconds to run.
Running subcircuit 2...
Subcircuit 2 took 20.65148687362671 seconds to run.
Running subcircuit 3...
Subcircuit 3 took 24.904088258743286 seconds to run.
Running subcircuit 4...
Subcircuit 4 took 32.534209966659546 seconds to run.
Running subcircuit 5...
Subcircuit 5 took 19.189459085464478 seconds to run.
Running original circuit...


/var/folders/w5/qtgfh_kx375_3385y0kbwgy40000gn/T/ipykernel_6724/3653426976.py:112: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/api/migration-guides/qiskit-runtime.
  original_counts = backend.run(circuit).result().get_counts()


RuntimeJobFailureError: 'Unable to retrieve job result. Instruction s on qubits (0,) from the 0-th circuit is not supported by the target. -- \\n        Transpile your circuits for the target before submitting a primitive query. For\\n        example, you can use the following code block given an IBMBackend object `backend`\\n        and circuits of type `List[QuantumCircuit]`:\\n            from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager\\n            pm = generate_preset_pass_manager(optimization_level=1, target=backend.target)\\n            isa_circuits = pm.run(circuits)\\n        Then pass `isa_circuits` to the Sampler or Estimator.\\n         -- https://ibm.biz/error_codes#1517'